In [ ]:
# DNN classifier using the in box tensorflow classifier

import numpy as np
import tensorflow as tf
import time
import glob

def main():
    st = time.time()
    # get the data
    data_folder_path = "/Users/ssk4/Downloads/data_subimage/"
    files = glob.glob(data_folder_path + "*.npy")

    # shuffling the files to avoid any single dot bias
    import random
    random.shuffle(files)
    
    # downsample the input size for testing
    files = files[:1000]

    n_samples = len(files)
    train_sample_ratio = 0.8
    n_train = int(train_sample_ratio * n_samples)

    print("Total number of samples :",n_samples)
    print("Training samples :",n_train)
    print("Test samples :",n_samples - n_train)
    
    
    feature_columns = [tf.contrib.layers.real_valued_column("",dimension=4)]
    
    classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                               hidden_units=[1024,512,16],
                                               n_classes=4,
                                               optimizer=tf.train.ProximalAdagradOptimizer(
                                               learning_rate=0.1,
                                               l1_regularization_strength=0.001)
                                               )
    
    def get_train_inputs():
        n_batch = 50
        index = np.random.choice(n_train,n_batch,replace=False)
        inp = []
        oup = []
        for i in index:
            dat = np.load(files[i])
            inp += [dat.item()['current_map'].flatten()]
            oup += [dat.item()['label']]

        inp = np.array(inp,dtype=np.float32)
        oup = np.array(oup,dtype=np.float32)

        x = tf.constant(inp)
        y = tf.argmax(tf.constant(oup),axis=1)
        return x,y
    
    # Fit the model. Magic happens here!
    classifier.fit(input_fn=get_train_inputs, steps=50000)
    
    def get_test_inputs():
        inp = []
        oup = []
        for file in files[n_train:]:
            dat = np.load(file)
            inp += [dat.item()['current_map'].flatten()]
            oup += [dat.item()['label']]

        inp = np.array(inp,dtype=np.float32)
        oup = np.array(oup,dtype=np.float32)

        x = tf.constant(inp)
        y = tf.argmax(tf.constant(oup),axis=1)
        return x,y
    
    accuracy_score = classifier.evaluate(input_fn=get_train_inputs,steps=1)["accuracy"]
    print("Train accuracy",accuracy_score)
    accuracy_score = classifier.evaluate(input_fn=get_test_inputs,steps=1)["accuracy"]
    print("Test accuracy",accuracy_score)
    
if __name__ == "__main__":
    main()

Total number of samples : 1000
Training samples : 800
Test samples : 200
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11f6e0d30>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supp